In [31]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
from pathlib import Path
from tbparse import SummaryReader


In [32]:
MODEL_TYPE='informer' # GRU or informer
SAVE_PATH = f"../results/ICMI24_{MODEL_TYPE}_results.md"
SEED=21
OVERWRITE=True
if OVERWRITE:
     open(SAVE_PATH, 'w').close()


In [33]:
def get_metric_results_from_eventfile(dirpath, metrics, SAVE_PATH="", title="", select="latest"):
    # select = "latest", or "lowest" or "highest"
    reader = SummaryReader(dirpath, pivot=False, extra_columns={'dir_name'})
    df = reader.scalars
    df = df.rename(columns={"tag": "metric", "dir_name": "fold"})
    grouped = df.groupby(["fold","metric"])
    # ['step'].max().reset_index() # this is choosing the max AUROC but we want the latest!!
    if select=='latest':
        df = df.loc[grouped['step'].idxmax()].reset_index(drop=True)
    elif select=='highest':
        df = df.loc[grouped['value'].idxmax()].reset_index(drop=True)
    elif select=='lowest':
        df = df.loc[grouped['value'].idxmin()].reset_index(drop=True)
    
    df = df[df["metric"].isin(metrics)].set_index(["fold"]).sort_values("metric").reset_index()

    res = df.pivot(index='fold', columns='metric', values='value')#.rename_axis(columns=None)
    if SAVE_PATH:
        with open(SAVE_PATH,'a') as f:
            f.write("\n\n")
            f.write(title)
            f.write("\n")
        res.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=True,floatfmt='.3f')
    return res

In [34]:

# Get pretraining task val metrics 
tasks=['fi','cl','pc','rc']
metric_names=['FI: AUROC','CL: Accuracy','PC: MSE (deg)','RC: MSE (deg)']
metric_select=['highest','highest','lowest','lowest'] # which value to use when multiple checkpoints avail
varname = 'val_{task}_metric'
metrics=[varname.format(**locals()) for task in tasks]

pretrain_summary=[]
for i,metric in enumerate(metrics):
    name=metric_names[i]
    res=get_metric_results_from_eventfile(f"../lightning_logs/2023/{MODEL_TYPE}_pretraining_seed{SEED}", [metric], SAVE_PATH=None, title=name, select="latest")
    pretrain_summary.append({"metric":name, 
                       "mean": res[[metric]].mean().item(),
                       "min": res[[metric]].min().item(),
                       "max": res[[metric]].max().item()
                       })
    


In [35]:
    
tit="Pretraining validation metrics: summarized"
with open(SAVE_PATH, "a") as f:
    f.write("\n\n")
    f.write(tit)
    f.write("\n")
tb = pd.DataFrame.from_records(pretrain_summary)
tb.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=False,floatfmt='.3f')

In [36]:
labels = ['SVT',
'Rote_X', 'Rote_Y', "Rote_Z", "Rote_D",
          'Inference_X', "Inference_Y","Inference_Z", "Inference_D", 
          "Deep_X", "Deep_Z",
          "MW"]

In [37]:
metrics = ["train_auroc", "val_auroc", "train_accuracy_epoch", "val_accuracy_epoch"]
classifier_names = ['informer_4task_encoder_varlen_meanpool', 'informer_4task_encoder_varlen_maskmeanpool','informer_4task_encoder_varlen_finalpos']

for classifier in classifier_names:
    val_aurocs = []
    for label in labels:
        # tit=f"# {label}"
        # with open(SAVE_PATH, "a") as f:
        #     f.write("\n\n")
        #     f.write(tit)
        #     f.write("\n")
        res=get_metric_results_from_eventfile(f"../lightning_logs/2024/classifiers/{classifier}/{label}_{MODEL_TYPE}", metrics, SAVE_PATH=None, select="latest")
        res['label']=label
        val_aurocs.append({"label":label, 
                        "mean AUROC": res[['val_auroc']].mean().item(),
                        "min AUROC": res[['val_auroc']].min().item(),
                        "max AUROC": res[['val_auroc']].max().item()
                        })
    tit=f"Held-out AUROCs: {classifier}"
    with open(SAVE_PATH, "a") as f:
        f.write("\n\n")
        f.write(tit)
        f.write("\n")
    tb = pd.DataFrame.from_records(val_aurocs)
    tb.to_markdown(SAVE_PATH, mode="a", tablefmt="github", index=False,floatfmt='.3f')
